In [ ]:
import pyodbc
from datetime import datetime
from collections import defaultdict

# MES and ERP connection strings (Windows Auth)
MES_CONN_STR = (
    "DRIVER={SQL Server};SERVER=localhost\\SQLEXPRESSBK;"
    "DATABASE=MES;Trusted_Connection=yes;"
)
ERP_CONN_STR = (
    "DRIVER={SQL Server};SERVER=localhost\\SQLEXPRESSBK;"
    "DATABASE=ERP;Trusted_Connection=yes;"
)

def get_completed_batches():
    """
    Pull all completed production batches from MES.
    These are candidates to update ERP work orders.
    """
    with pyodbc.connect(MES_CONN_STR) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT WorkOrderID, QuantityProduced, EndTime
            FROM dbo.ProductionBatches
            WHERE Status = 'Completed'
        """)
        return cursor.fetchall()

def consolidate_batches(batches):
    """
    Aggregate MES batch records by WorkOrderID.
    - Sums quantity produced
    - Keeps latest EndTime
    - Tracks count (for duplicate detection)
    """
    consolidated = {}
    for batch in batches:
        wo_id = batch.WorkOrderID
        qty = batch.QuantityProduced
        end_time = batch.EndTime or datetime.now()

        if wo_id in consolidated:
            consolidated[wo_id]['total_qty'] += qty
            if end_time > consolidated[wo_id]['latest_end_time']:
                consolidated[wo_id]['latest_end_time'] = end_time
            consolidated[wo_id]['count'] += 1
        else:
            consolidated[wo_id] = {
                'total_qty': qty,
                'latest_end_time': end_time,
                'count': 1
            }
    return consolidated

def fetch_erp_work_order(cursor, work_order_id):
    """
    Look up ERP work order to validate status and compare quantities.
    """
    cursor.execute("""
        SELECT QuantityCompleted, QuantityPlanned, Status
        FROM dbo.WorkOrders
        WHERE WorkOrderID = ?
    """, work_order_id)
    return cursor.fetchone()

def update_work_orders(consolidated_data):
    """
    Apply consolidated MES data to ERP work orders:
    - Skip already completed or fully matched orders
    - Update completed quantity and completion date
    """
    with pyodbc.connect(ERP_CONN_STR) as conn:
        cursor = conn.cursor()
        for wo_id, data in consolidated_data.items():
            total_qty = data['total_qty']
            latest_end_time = data['latest_end_time']
            count = data['count']

            erp_order = fetch_erp_work_order(cursor, wo_id)
            if not erp_order:
                print(f"❌ Work Order {wo_id} not found in ERP.")
                continue

            qty_completed, qty_planned, status = erp_order

            # Skip if already complete or quantity is fully accounted for
            if status == 'Completed' or (qty_completed or 0) >= qty_planned:
                print(f"⚠️ Skipping Work Order {wo_id} – Already completed or fully matched.")
                continue

            print(f"[✔] Updating Work Order {wo_id} from {count} MES record(s):")
            print(f"     +{total_qty} units → CompletionDate = {latest_end_time}")

            cursor.execute("""
                UPDATE dbo.WorkOrders
                SET
                    QuantityCompleted = ISNULL(QuantityCompleted, 0) + ?,
                    Status = 'Completed',
                    CompletionDate = ?
                WHERE WorkOrderID = ?
            """, total_qty, latest_end_time, wo_id)

        conn.commit()
        print("\n✅ ERP work orders updated with consolidated MES quantities.")

def report_duplicates(consolidated_data):
    """
    Log any WorkOrderIDs that had more than one completed batch in MES.
    """
    duplicates = {wo: data['count'] for wo, data in consolidated_data.items() if data['count'] > 1}
    if duplicates:
        print("\n🔁 Duplicate MES entries detected:")
        for wo, count in duplicates.items():
            print(f"⚠️  WorkOrderID {wo} has {count} MES batches")
    else:
        print("\n✅ No duplicate MES entries found.")

def main():
    print("🔍 Retrieving completed MES batches...")
    completed_batches = get_completed_batches()

    if not completed_batches:
        print("⚠️ No completed MES batches found.")
        return

    consolidated = consolidate_batches(completed_batches)
    print(f"📦 Consolidated {len(consolidated)} unique WorkOrder(s) from {len(completed_batches)} batch record(s).")

    report_duplicates(consolidated)
    update_work_orders(consolidated)

if __name__ == "__main__":
    main()
